In [273]:
import os
import sys
from collections import defaultdict

import importlib
import abxportinf
import abxportinf._grouper
import abxportinf._optimize
import abxportinf._vectorizer
import abxportinf.busdef
import abxportinf.main 
import abxportinf.util
from abxportinf.busdef import BusDef

modules = [
    '_grouper',
    '_optimize',
    '_vectorizer',
    'busdef',
    'main', 
    'util',
]
importlib.reload(abxportinf)
for mod in modules:
    importlib.reload(getattr(abxportinf, mod))


In [92]:
block_root = '/ddr'
comp_json_path = os.path.join(block_root, 'ddr.json5')
ports = abxportinf.get_ports_from_json5(comp_json_path)
#abxportinf.get_bus_matches(None, None)
#abxportinf.get_bus_defs(None)

Warning ('regHADDR', 'PAR_REG_ADDR_WIDTH + 2') not correctly parsed
Warning ('phy_reg_addr', '-(PAR_REG_ADDR_WIDTH)') not correctly parsed
296


In [136]:
pg, Z, wire_names = abxportinf._grouper.get_port_grouper(ports)

#port = ports[34]
#for i, group in enumerate(pg.get_port_groups(port)):
#    print('group {}, size: {}'.format(i, len(group)))                                                                                                                                                                                   
#    for wire in group:                                
#        print('  - ', wire)                           
         
phy = [   
    ('ready1', 1, 1),
    ('ready2', 1, 1),
    ('ready3', 1, 1),
    #('xxxx', 1, 1),            
    #('yyyx', 1, 1),                        
]
bus = [            
    ('rdy3', 1, 1),                                   
    ('rdy1', 1, 1),
    ('rdy2', 1, 1),                    
]                                             
mapping = abxportinf._optimize.map_port_sets(phy, bus)
           
assn_ports = set()
                       
bus_def = [           
    ('dfi_wrlvl_strobe', 9, -1),
    ('dfi_wrlvl_en', 1, 1),
    ('dfi_wrlvl_resp', 1, 1),
]               
for ii, port in enumerate(ports):
    # only assign ports to a single bus definition
    if port in assn_ports: 
        continue
    mappings = []
    #print('root port', port)
    ccost = None
    for i, port_group in enumerate(pg.get_port_groups(port)):
        mapping, cost = abxportinf._optimize.map_port_sets(port_group, bus_def)
        mappings.append((cost, port_group, mapping))
        #print('{} size of port group {}, cost {}'.format(i, len(port_group), cost))
        if ccost and cost > ccost:
            break
        ccost = cost
    cost, cport_group, cmapping = min(mappings)
    assn_ports |= set(cport_group)
    

In [169]:
bus_spec_rootdir = '/bus-defs/specs'
                                
print('loading bus defs from specs')
bus_defs = []                
for root, dirs, fnames in os.walk(bus_spec_rootdir):
    #print((root, dirs, files))  
    for fname in fnames:             
        spec_path = os.path.join(root, fname)
        if BusDef.is_spec_bus_def(spec_path):     
            print('  - loading ', spec_path)
            bus_defs.extend(
                BusDef.bus_defs_from_spec(spec_path)
            )                
print('  - done')                                            
print('loaded {} bus definitions from specs'.format(len(bus_defs)))
print('  - total req ports', sum([bd.num_req_ports for bd in bus_defs]))
print('  - total opt ports', sum([bd.num_opt_ports for bd in bus_defs]))

spec_path = '/bus-defs/specs/amba.com/AMBA4/AXI4/r0p0_0/AXI4_rtl.json5'
bus_defs = BusDef.bus_defs_from_spec(spec_path)

loading bus defs from specs
  - loading  /bus-defs/specs/amba.com/AMBA2/AHB/r3p0_1/AHB_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA5/AHB5Target/r0p0_0/AHB5Target_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA5/AHB5Initiator/r0p0_0/AHB5Initiator_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/AXI4Stream/r0p0_1/AXI4Stream_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/AXI4/r0p0_0/AXI4_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/AXI4/r0p0_0/AXI4_RO_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/ACE-Lite/r0p0_0/ACE-Lite_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/APB4/r0p0_0/APB4_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/ATB/r0p0_0/ATB_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/ACP/r0p0_0/ACP_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA4/ACE/r0p0_0/ACE_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA3/AHBLite/r2p0_0/AHBLite_rtl.json5
  - loading  /bus-defs/specs/amba.com/AMBA3/APB/r2p0_0/APB_rtl.json5


In [275]:
bus_mappings = abxportinf.get_bus_matches(ports, bus_defs)
cost, bus_def, bus_mapping = min(bus_mappings)
print(bus_mapping)

{('axi0_WLAST', 1, 1): ('WLAST', 1, 1), ('axi0_AWQOS', 1, 1): ('AWQOS', 4, 1), ('axi0_ARQOS', 1, 1): ('ARQOS', 4, 1), ('axi0_AWLEN', 8, 1): ('AWLEN', 8, 1), ('axi0_AWSIZE', 3, 1): ('AWSIZE', 3, 1), ('axi0_WDATA', 256, 1): ('WDATA', None, 1), ('axi0_AWAPCMD', 1, 1): ('ARLOCK', 1, 1), ('axi0_AWCOBUF', 1, 1): ('ARLOCK', 1, 1), ('axi0_AWADDR', 37, 1): ('AWADDR', None, 1), ('axi0_AWBURST', 2, 1): ('ARBURST', 2, 1), ('axi0_WSTRB', 32, 1): ('WSTRB', None, 1), ('axi0_AWVALID', 1, 1): ('ARVALID', 1, 1), ('axi0_WVALID', 1, 1): ('WVALID', 1, 1), ('axi0_ARESETn', 1, 1): ('ARESETn', 1, -1), ('axi0_AWPROT', 2, 1): ('ARBURST', 2, 1), ('axi0_AWID', 12, 1): ('AWID', None, 1), ('axi0_AWLOCK', 1, 1): ('AWLOCK', 1, 1), ('axi0_ACLK', 1, 1): ('ACLK', 1, -1), ('axi0_BREADY', 1, 1): ('BREADY', 1, 1), ('axi0_AWALLSTRB', 1, 1): ('RREADY', 1, 1), ('axi0_AWREGION', 2, 1): ('AWREGION', 4, 1)}
